In [ ]:
import argparse, os, logging, random, time
import numpy as np
import math
import time
import scipy.sparse
import lightgbm as lgb
import data_helpers as dh

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from sklearn.utils.extmath import softmax

from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import Optimizer, AdamW

import gc

In [ ]:
torch.__version__

'1.10.0+cu102'

In [ ]:
torchvision.__version__

'0.11.1+cu102'

In [ ]:
import pdb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    type_prefix = torch.cuda
else:
    type_prefix = torch

In [ ]:
from importlib import reload 

reload(logging)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
# create file handler which logs even debug messages
fh = logging.FileHandler('wine-100-emb.log')
fh.setLevel(logging.INFO)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)

In [ ]:
def one_hot(y, numslot, mask=None):
    y_tensor = y.type(type_prefix.LongTensor).reshape(-1, 1)
    y_one_hot = torch.zeros(y_tensor.size()[0], numslot, device=device, dtype=torch.float32, requires_grad=False).scatter_(1, y_tensor, 1)
    if mask is not None:
        y_one_hot = y_one_hot * mask
    y_one_hot = y_one_hot.reshape(y.shape[0], -1)
    return y_one_hot

In [ ]:
class BatchDense(nn.Module):
    def __init__(self, batch, in_features, out_features, bias_init=None):
        super(BatchDense, self).__init__()
        self.batch = batch
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(batch, in_features, out_features))
        self.bias = Parameter(torch.Tensor(batch, 1, out_features))
        self.reset_parameters(bias_init)
    def reset_parameters(self, bias_init=None):
        stdv = math.sqrt(6.0 /(self.in_features + self.out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if bias_init is not None:
            self.bias.data = torch.from_numpy(bias_init)
        else:
            self.bias.data.fill_(0)
    def forward(self, x):
        size = x.size()
        # Todo: avoid the swap axis
        x = x.view(x.size(0), self.batch, -1)
        out = x.transpose(0, 1).contiguous()
        out = torch.baddbmm(self.bias, out, self.weight)
        out = out.transpose(0, 1).contiguous()
        out = out.view(x.size(0), -1)
        return out

In [ ]:
class EmbeddingModel(nn.Module):
    def __init__(self, n_models, max_ntree_per_split, embsize, maxleaf, n_output, out_bias=None, task='regression'):
        super(EmbeddingModel, self).__init__()
        self.task = task
        self.n_models = n_models
        self.maxleaf = maxleaf
        self.fcs = nn.ModuleList()
        self.max_ntree_per_split = max_ntree_per_split

        self.embed_w = Parameter(torch.Tensor(n_models, max_ntree_per_split*maxleaf, embsize))
        # torch.nn.init.xavier_normal(self.embed_w)
        stdv = math.sqrt(1.0 /(max_ntree_per_split))
        self.embed_w.data.normal_(0,stdv) # .uniform_(-stdv, stdv)
        
        self.bout = BatchDense(n_models, embsize, 1, out_bias)
        self.bn = nn.BatchNorm1d(embsize * n_models)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        # self.output_fc = Dense(n_models * embsize, n_output)
        self.dropout = torch.nn.Dropout()
        if task == 'regression':
            self.criterion = nn.MSELoss()
        else:
            self.criterion = nn.BCELoss()

    def batchmul(self, x, models, embed_w, length):
        out = one_hot(x, length)
        out = out.view(x.size(0), models, -1)
        out = out.transpose(0, 1).contiguous()
        out = torch.bmm(out, embed_w)
        out = out.transpose(0, 1).contiguous()
        out = out.view(x.size(0), -1)
        return out
        
    def lastlayer(self, x):
        out = self.batchmul(x, self.n_models, self.embed_w, self.maxleaf)
        out = self.bn(out)
        # out = self.tanh(out)
        # out = out.view(x.size(0), self.n_models, -1)
        return out
    def forward(self, x):
        out = self.lastlayer(x)
        out = self.dropout(out)
        out = out.view(x.size(0), self.n_models, -1)
        out = self.bout(out)
        # out = self.output_fc(out)
        sum_out = torch.sum(out,-1,True)
        if self.task != 'regression':
            return self.sigmoid(sum_out), out
        return sum_out, out
    
    def joint_loss(self, out, target, out_inner, target_inner, *args):
        return nn.MSELoss()(out_inner, target_inner)

    def true_loss(self, out, target):
        return self.criterion(out, target)

In [ ]:
def eval_metrics(task, true, pred):
    if task == 'binary':
        logloss = sklearn.metrics.log_loss(true.astype(np.float64), pred.astype(np.float64))
        auc = sklearn.metrics.roc_auc_score(true, pred)
        # error = 1-sklearn.metrics.accuracy_score(true,(pred+0.5).astype(np.int32))
        return (logloss, auc)#, error)
    else:
        mseloss = sklearn.metrics.mean_squared_error(true, pred)
        return mseloss

def EvalTestset(test_x, test_y, model, test_batch_size, test_x_opt=None):
    test_len = test_x.shape[0]
    test_num_batch = math.ceil(test_len / test_batch_size)
    sum_loss = 0.0
    y_preds = []
    model.eval()
    with torch.no_grad():
        for jdx in range(test_num_batch):
            tst_st = jdx * test_batch_size
            tst_ed = min(test_len, tst_st + test_batch_size)
            inputs = torch.from_numpy(test_x[tst_st:tst_ed].astype(np.float32)).to(device)
            if test_x_opt is not None:
                inputs_opt = torch.from_numpy(test_x_opt[tst_st:tst_ed].astype(np.float32)).to(device)
                outputs = model(inputs, inputs_opt)
            else:
                outputs = model(inputs)
            targets = torch.from_numpy(test_y[tst_st:tst_ed]).to(device)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            y_preds.append(outputs)
            loss_tst = model.true_loss(outputs, targets).item()
            sum_loss += (tst_ed - tst_st) * loss_tst
    return sum_loss / test_len, np.concatenate(y_preds, 0)

def TrainWithLog(loss_dr, loss_init, loss_de, log_freq, test_freq, task, test_batch_size,                
                train_x, train_y, 
                 train_y_opt, test_x, test_y, model, opt,
                 epoch, batch_size, n_output, key="",
                 train_x_opt=None, test_x_opt=None):
    # trn_writer = tf.summary.FileWriter(summaryPath+plot_title+key+"_output/train")
    # tst_writer = tf.summary.FileWriter(summaryPath+plot_title+key+"_output/test")
    if isinstance(test_x, scipy.sparse.csr_matrix):
        test_x = test_x.todense()
    train_len = train_x.shape[0]
    global_iter = 0
    trn_batch_size = batch_size
    train_num_batch = math.ceil(train_len / trn_batch_size)
    total_iterations = epoch * train_num_batch
    start_time = time.time()
    total_time = 0.0
    min_loss = float("Inf")
    # min_error = float("Inf")
    max_auc = 0.0
    for epoch in range(epoch):
        shuffled_indices = np.random.permutation(np.arange(train_x.shape[0]))
        Loss_trn_epoch = 0.0
        Loss_trn_log = 0.0
        log_st = 0
        for local_iter in range(train_num_batch):
            trn_st = local_iter * trn_batch_size
            trn_ed = min(train_len, trn_st + trn_batch_size)
            batch_trn_x = train_x[shuffled_indices[trn_st:trn_ed]]
            if isinstance(batch_trn_x, scipy.sparse.csr_matrix):
                batch_trn_x = batch_trn_x.todense()
            inputs = torch.from_numpy(batch_trn_x.astype(np.float32)).to(device)
            targets = torch.from_numpy(train_y[shuffled_indices[trn_st:trn_ed],:]).to(device)
            model.train()
            if train_x_opt is not None:
                inputs_opt = torch.from_numpy(train_x_opt[shuffled_indices[trn_st:trn_ed]].astype(np.float32)).to(device)
                outputs = model(inputs, inputs_opt)
            else:
                outputs = model(inputs)
            opt.zero_grad()
            if isinstance(outputs, tuple) and train_y_opt is not None:
                # targets_inner = torch.from_numpy(s_train_y_opt[trn_st:trn_ed,:]).to(device)
                targets_inner = torch.from_numpy(train_y_opt[shuffled_indices[trn_st:trn_ed],:]).to(device)
                loss_ratio = loss_init * max(0.3,loss_dr ** (epoch // loss_de))#max(0.5, args.loss_dr ** (epoch // args.loss_de))
                if len(outputs) == 3:
                    loss_val = model.joint_loss(outputs[0], targets, outputs[1], targets_inner, loss_ratio, outputs[2])
                else:
                    loss_val = model.joint_loss(outputs[0], targets, outputs[1], targets_inner, loss_ratio)
                loss_val.backward()
                loss_val = model.true_loss(outputs[0], targets)
            elif isinstance(outputs, tuple):
                loss_val = model.true_loss(outputs[0], targets)
                loss_val.backward()
            else:
                loss_val = model.true_loss(outputs, targets)
                loss_val.backward()
            opt.step()
            loss_val = loss_val.item()
            global_iter += 1
            Loss_trn_epoch += (trn_ed - trn_st) * loss_val
            Loss_trn_log += (trn_ed - trn_st) * loss_val
            if global_iter % log_freq == 0:
                print(key+"Epoch-{:0>3d} {:>5d} Batches, Step {:>6d}, Training Loss: {:>9.6f} (AllAvg {:>9.6f})"
                            .format(epoch, local_iter + 1, global_iter, Loss_trn_log/(trn_ed-log_st), Loss_trn_epoch/trn_ed))
                
                # trn_summ = tf.Summary()
                # trn_summ.value.add(tag=args.data+ "/Train/Loss", simple_value = Loss_trn_log/(trn_ed-log_st))
                # trn_writer.add_summary(trn_summ, global_iter)
                log_st = trn_ed
                Loss_trn_log = 0.0
            if global_iter % test_freq == 0 or local_iter == train_num_batch - 1:
                if model == 'deepgbm' or model == 'd1':
                    try:
                        print('Alpha: '+str(model.alpha))
                        print('Beta: '+str(model.beta))
                    except:
                        pass
                # tst_summ = tf.Summary()
                torch.cuda.empty_cache()
                test_loss, pred_y = EvalTestset(test_x, test_y, model, test_batch_size, test_x_opt)
                current_used_time = time.time() - start_time
                start_time = time.time()
                total_time += current_used_time
                remaining_time = (total_iterations - (global_iter) ) * (total_time / (global_iter))
                if task == 'binary':
                    metrics = eval_metrics(task, test_y, pred_y)
                    _, test_auc = metrics
                    # min_error = min(min_error, test_error)
                    max_auc = max(max_auc, test_auc)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/Error", simple_value = test_error)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/AUC", simple_value = test_auc)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/Min_Error", simple_value = min_error)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/Max_AUC", simple_value = max_auc)
                    print(key+"Evaluate Result:\nEpoch-{:0>3d} {:>5d} Batches, Step {:>6d}, Testing Loss: {:>9.6f}, Testing AUC: {:8.6f}, Used Time: {:>5.1f}m, Remaining Time: {:5.1f}m"
                            .format(epoch, local_iter + 1, global_iter, test_loss, test_auc, total_time/60.0, remaining_time/60.0))
                else:
                    print(key+"Evaluate Result:\nEpoch-{:0>3d} {:>5d} Batches, Step {:>6d}, Testing Loss: {:>9.6f}, Used Time: {:>5.1f}m, Remaining Time: {:5.1f}m"
                            .format(epoch, local_iter + 1, global_iter, test_loss, total_time/60.0, remaining_time/60.0))
                min_loss = min(min_loss, test_loss)
                # tst_summ.value.add(tag=args.data+"/Test/Loss", simple_value = test_loss)
                # tst_summ.value.add(tag=args.data+"/Test/Min_Loss", simple_value = min_loss)
                print("-------------------------------------------------------------------------------")
                # tst_writer.add_summary(tst_summ, global_iter)
                # tst_writer.flush()
        print("Best Metric: %s"%(str(max_auc) if task=='binary' else str(min_loss)))
        print("####################################################################################")
    print("Final Best Metric: %s"%(str(max_auc) if task=='binary' else str(min_loss)))
    return min_loss        

def GetEmbPred(model, fun, X, test_batch_size):
    model.eval()
    tst_len = X.shape[0]
    test_num_batch = math.ceil(tst_len / test_batch_size)
    y_preds = []
    with torch.no_grad():
        for jdx in range(test_num_batch):
            tst_st = jdx * test_batch_size
            tst_ed = min(tst_len, tst_st + test_batch_size)
            inputs = torch.from_numpy(X[tst_st:tst_ed]).to(device)
            t_preds = fun(inputs).data.cpu().numpy()
            y_preds.append(t_preds)
        y_preds = np.concatenate(y_preds, 0)
    return y_preds


In [ ]:
HOME_DIR = os.getcwd()
DATA_DIR = os.path.join(HOME_DIR, 'data')

In [ ]:
num_data = dh.load_data('/work/neurotrees/articles code reproduction/DeepGBM/data/data_offline_num')

In [ ]:
train_x, train_y, test_x, test_y = num_data

In [ ]:
import pickle
with open('n_models_wine_100.pickle', 'rb') as f:
    # Pickle using the highest protocol available.
    n_models = pickle.load(f)
    
with open('max_ntree_per_split_wine_100.pickle', 'rb') as f:
    # Pickle using the highest protocol available.
    max_ntree_per_split = pickle.load(f)
    
with open('group_average_wine_100.pickle', 'rb') as f:
    # Pickle using the highest protocol available.
    group_average = pickle.load(f)

with open('leaf_preds_wine_100.pickle', 'rb') as f:
    # Pickle using the highest protocol available.
    leaf_preds = pickle.load(f)
    
with open('test_leaf_preds_wine_100.pickle', 'rb') as f:
    # Pickle using the highest protocol available.
    test_leaf_preds = pickle.load(f)
    
with open('tree_outputs_wine_100.pickle', 'rb') as f:
    # Pickle using the highest protocol available.
    tree_outputs = pickle.load(f) 
    

In [ ]:
embsize = 20
maxleaf = 64
task = "regression"
l2_reg = 1e-6
emb_lr = 1e-3
emb_epoch = 2
batch_size = 512
test_batch_size = 100 
loss_init = 1.0
loss_dr = 0.7
loss_de = 2
log_freq = 500
test_freq = 300
key = ""

n_output = train_y.shape[1]



emb_model = EmbeddingModel(n_models, max_ntree_per_split, embsize, maxleaf+1, n_output,
                           group_average, task=task).to(device)

opt = AdamW(emb_model.parameters(), lr=emb_lr, weight_decay=l2_reg)
tree_outputs = np.asarray(tree_outputs).reshape((n_models, leaf_preds.shape[0])).transpose((1,0))

TrainWithLog(loss_dr, loss_init, loss_de, log_freq, test_freq, task, test_batch_size,
             leaf_preds, train_y, tree_outputs,
                test_leaf_preds, test_y, emb_model, opt,
                emb_epoch, batch_size, n_output, key+"emb-")


output_w = emb_model.bout.weight.data.cpu().numpy().reshape(n_models*embsize, n_output)
output_b = np.array(emb_model.bout.bias.data.cpu().numpy().sum())
train_embs = GetEmbPred(emb_model, emb_model.lastlayer, leaf_preds, test_batch_size)
del tree_outputs, leaf_preds, test_leaf_preds
gc.collect();


emb-Evaluate Result:
Epoch-000     8 Batches, Step      8, Testing Loss: 33.951863, Used Time:   0.0m, Remaining Time:   0.0m
-------------------------------------------------------------------------------
Best Metric: 33.95186288016183
####################################################################################
emb-Evaluate Result:
Epoch-001     8 Batches, Step     16, Testing Loss: 32.910239, Used Time:   0.0m, Remaining Time:   0.0m
-------------------------------------------------------------------------------
Best Metric: 32.910239239128266
####################################################################################
Final Best Metric: 32.910239239128266


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=de072003-a9db-4342-8067-19a4b45feff1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>